In [ ]:
!pip install -q tensorflow-addons==0.20.0
!pip install -q git+https://github.com/hoyso48/tf-utils@main
!pip install -q pyarrow leven

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyarrow.parquet as pq
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as k
import tensorflow_addons as tfa
from tf_utils.schedules import OneCycleLR

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import time
import json
import random

tqdm.pandas()

print(f'Tensorflow Version {tf.__version__}')
print(f'Python Version: {sys.version}')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Tensorflow Version 2.12.0
Python Version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]


## Configs

In [ ]:
CONFIG = {"exp_name": 'test',
          "device": "GPU", # CPU, GPU, TPU-VM
          "max_len" : 128,
          "epoch": 50,
          "valid_size": 0.1,
          "supplemental_size": 0.5,
          "batch_size": 128,
          "optimizer": 'ranger', # 'ranger', 'adam'
          "loss": 'ctc', # 'ctc', 'cce'
          "lr": 1e-3,
          "lr_min": 5e-7,
          "weight_decay": 0.1,
          "warmup": 0}

!mkdir output
root = '/content'
root_save = '/content/output'
print(CONFIG["exp_name"])

test


In [ ]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def get_strategy(device='TPU-VM'):
    IS_TPU = False
    if "TPU" in device:
        tpu = 'local' if device=='TPU-VM' else None
        print("connecting to TPU...")
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        IS_TPU = True

    if device == "GPU"  or device == "CPU":
        ngpu = len(tf.config.experimental.list_physical_devices('GPU'))
        if ngpu>1:
            print("Using GPU")
            strategy = tf.distribute.MirroredStrategy()
        elif ngpu==1:
            print("Using single GPU")
            strategy = tf.distribute.get_strategy()
        else:
            print("Using CPU")
            strategy = tf.distribute.get_strategy()

    if device == "GPU":
        print("Num GPUs Available: ", ngpu)

    AUTO     = tf.data.experimental.AUTOTUNE
    REPLICAS = strategy.num_replicas_in_sync
    print(f'REPLICAS: {REPLICAS}')

    return strategy, REPLICAS, IS_TPU

STRATEGY, N_REPLICAS, IS_TPU = get_strategy(CONFIG["device"])
CONFIG["batch_size"] = CONFIG["batch_size"] * N_REPLICAS
CONFIG["lr"] = CONFIG["lr"] * N_REPLICAS

Using CPU
Num GPUs Available:  0
REPLICAS: 1


## Process Data

In [ ]:
GCS_PATH = {'asl-fingerspelling':'gs://kds-838fb40869e092ab9936d6e90a7b7c43c26bb07471fcd3be7d29707a',
            'asl-tfr':'gs://kds-26a3f077e7100aecb626e6fecf52e9044e6382c5b31f0caf36be9290',
            'asl-tfr-supplemental':'gs://kds-ef88fb86efbd26aba9ea51c72e42e39f8868f1b95529eae69e51701b'}

TRAIN_FILENAMES = tf.io.gfile.glob(GCS_PATH['asl-tfr']+'/train/*.tfrecord')
TRAIN_FILENAMES_SUPPLEMENTAL = tf.io.gfile.glob(GCS_PATH['asl-tfr-supplemental']+'/supplemental/*.tfrecord')
COMPETITION_PATH = GCS_PATH['asl-fingerspelling']

print("Train Files: ", len(TRAIN_FILENAMES))
print("Supplemental Train Files: ", len(TRAIN_FILENAMES_SUPPLEMENTAL))
!gsutil cp {COMPETITION_PATH}/train.csv .
!gsutil cp {COMPETITION_PATH}/supplemental_metadata.csv .
!gsutil cp {COMPETITION_PATH}/character_to_prediction_index.json .

Train Files:  68
Supplemental Train Files:  53
CommandException: No URLs matched: gs://kds-7a1de6f6fb09cdad50ebab8364ce5c9e2937606d5437de561aab1448/train.csv
CommandException: No URLs matched: gs://kds-7a1de6f6fb09cdad50ebab8364ce5c9e2937606d5437de561aab1448/supplemental_metadata.csv
CommandException: No URLs matched: gs://kds-7a1de6f6fb09cdad50ebab8364ce5c9e2937606d5437de561aab1448/character_to_prediction_index.json


In [ ]:
train = pd.read_csv(f'{root}/train.csv')
train_supplemental = pd.read_csv(f'{root}/supplemental_metadata.csv')
NUM_DATA = len(train)

FileNotFoundError: ignored

In [ ]:
LIP = [
    61, 185, 40, 39, 37, 267, 269, 270, 409, 291,
    146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]

NOSE = [1, 2, 98, 327]

LEYE = [466, 388, 387, 386, 385, 384, 398,
        263, 249, 390, 373, 374, 380, 381, 382, 362]

REYE = [246, 161, 160, 159, 158, 157, 173,
        33, 7, 163, 144, 145, 153, 154, 155, 133]

LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

LIP_ = [f'x_face_{i}' for i in LIP] + [f'y_face_{i}' for i in LIP] + [f'z_face_{i}' for i in LIP]
NOSE_ = [f'x_face_{i}' for i in NOSE] + [f'y_face_{i}' for i in NOSE] + [f'z_face_{i}' for i in NOSE]
LEYE_ = [f'x_face_{i}' for i in LEYE] + [f'y_face_{i}' for i in LEYE] + [f'z_face_{i}' for i in LEYE]
REYE_ = [f'x_face_{i}' for i in REYE] + [f'y_face_{i}' for i in REYE] + [f'z_face_{i}' for i in REYE]

LHAND_ = [f'x_left_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)]
RHAND_ = [f'x_right_hand_{i}' for i in range(21)] + [f'y_right_hand_{i}' for i in range(21)] + [f'z_right_hand_{i}' for i in range(21)]
POSE_ = [f'x_pose_{i}' for i in POSE] + [f'y_pose_{i}' for i in POSE] + [f'z_pose_{i}' for i in POSE]

FEATURE_COL = LIP_ + NOSE_ + LEYE_ + REYE_ + LHAND_ + RHAND_ + POSE_
FRAME_LEN = 128

In [ ]:
X_IDX = [i for i, col in enumerate(FEATURE_COL) if "x_" in col]
Y_IDX = [i for i, col in enumerate(FEATURE_COL) if "y_" in col]
Z_IDX = [i for i, col in enumerate(FEATURE_COL) if "z_" in col]

RHAND_IDX = [i for i, col in enumerate(FEATURE_COL) if "right_hand" in col]
LHAND_IDX = [i for i, col in enumerate(FEATURE_COL) if "left_hand" in col]
RPOSE_IDX = [i for i, col in enumerate(FEATURE_COL) if "pose" in col and int(col.split("_")[-1]) in RPOSE]
LPOSE_IDX = [i for i, col in enumerate(FEATURE_COL) if "pose" in col and int(col.split("_")[-1]) in LPOSE]
LIP_IDX = [i for i, col in enumerate(FEATURE_COL) if "face" in col and int(col.split("_")[-1]) in LIP]
NOSE_IDX = [i for i, col in enumerate(FEATURE_COL) if "face" in col and int(col.split("_")[-1]) in NOSE]
LEYE_IDX = [i for i, col in enumerate(FEATURE_COL) if "face" in col and int(col.split("_")[-1]) in LEYE]
REYE_IDX = [i for i, col in enumerate(FEATURE_COL) if "face" in col and int(col.split("_")[-1]) in REYE]
# NORM_IDX = [i for i, col in enumerate(FEATURE_COL) if "pose" == col and int(col[-2:]) in LPOSE]

print(NOSE_IDX)

In [ ]:
with open(f'{root}/character_to_prediction_index.json') as json_file:
    CHAR2ORD = json.load(json_file)

# Character to Ordinal Encoding Mapping
# display(pd.Series(CHAR2ORD).to_frame('Ordinal Encoding'))

pad_token = '^'
PAD_TOKEN_IDX = 59
CHAR2ORD[pad_token] = PAD_TOKEN_IDX
ORD2CHAR = {j:i for i,j in CHAR2ORD.items()}
print(len(CHAR2ORD))

## Preprocessing
lots of todo

In [ ]:
## TODO

class Preprocess(tf.keras.layers.Layer):
    def __init__(self, max_len=128, **kwargs):
        super().__init__(**kwargs)
        self.max_len = max_len

    def call(self, inputs):
        return inputs

In [ ]:
# TODO
# still the same as starter
def resize_pad(x, max_len):
    if tf.shape(x)[0] < max_len:
        x = tf.pad(x, ([[0, max_len-tf.shape(x)[0]], [0, 0], [0, 0]]))
    else:
        x = tf.image.resize(x, (max_len, tf.shape(x)[1]))
    return x

## TODO
def augment_fn(x, max_len):
    return x

def stack_to_3d(x, IDX, norm=False):
    x_x = x[:, 0*(len(IDX)//3) : 1*(len(IDX)//3)]
    x_y = x[:, 1*(len(IDX)//3) : 2*(len(IDX)//3)]
    x_z = x[:, 2*(len(IDX)//3) : 3*(len(IDX)//3)]
    x = tf.concat([x_x[..., tf.newaxis], x_y[..., tf.newaxis], x_z[..., tf.newaxis]], axis=-1)
    if norm:
        mean = tf.math.reduce_mean(x, axis=1)[:, tf.newaxis, :]
        std = tf.math.reduce_std(x, axis=1)[:, tf.newaxis, :]
        x = (x - mean) / std
    return x

# Detect the dominant hand from the number of NaN values.
# Dominant hand will have less NaN values since it is in frame moving.
def preprocess(landmarks, phrase, augment=True, max_len=64, norm=False):
    x = landmarks
    # lip = tf.gather(x, tf.cast(LIP_IDX, tf.int32), axis=1)
    # nose = tf.gather(x, tf.cast(NOSE_IDX, tf.int32), axis=1)
    # leye = tf.gather(x, tf.cast(LEYE_IDX, tf.int32), axis=1)
    # reye = tf.gather(x, tf.cast(REYE_IDX, tf.int32), axis=1)
    rhand = tf.gather(x, tf.cast(RHAND_IDX, tf.int32), axis=1)
    lhand = tf.gather(x, tf.cast(LHAND_IDX, tf.int32), axis=1)
    rpose = tf.gather(x, tf.cast(RPOSE_IDX, tf.int32), axis=1)
    lpose = tf.gather(x, tf.cast(LPOSE_IDX, tf.int32), axis=1)

    # lip = stack_to_3d(lip, LIP_IDX, norm=norm)
    # nose = stack_to_3d(nose, NOSE_IDX, norm=norm)
    # leye = stack_to_3d(leye, LEYE_IDX, norm=norm)
    # reye = stack_to_3d(reye, REYE_IDX, norm=norm)
    rhand = stack_to_3d(rhand, RHAND_IDX, norm=norm)
    lhand = stack_to_3d(lhand, LHAND_IDX, norm=norm)
    rpose = stack_to_3d(rpose, RPOSE_IDX, norm=norm)
    lpose = stack_to_3d(lpose, LPOSE_IDX, norm=norm)

    x = tf.concat([rhand, lhand, rpose, lpose], axis=1)
    if augment:
        x = augment_fn(x, max_len=max_len)

    print(x.shape)
    x = resize_pad(x, max_len)

    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
    x = tf.reshape(x, (FRAME_LEN, len(LHAND_ + RHAND_ + POSE_)))
    return x, phrase

In [ ]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(CHAR2ORD.keys()),
        values=list(CHAR2ORD.values()),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
)

def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COL}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COL])
    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)

    return landmarks, phrase

def convert_fn(landmarks, phrase):
    # Add start and end pointers to phrase.
    phrase = tf.strings.bytes_split(phrase)
    phrase = table.lookup(phrase)
    # Vectorize and add padding.
    phrase = tf.pad(phrase, paddings=[[0, 64 - tf.shape(phrase)[0]]], mode = 'CONSTANT',
                    constant_values = PAD_TOKEN_IDX)
#     t1 = [SOS_TOKEN]
#     t2 = [EOS_TOKEN]
#     phrase = tf.concat([t1, phrase, t2], axis=0)

    return landmarks, phrase

In [ ]:
# see tfr data
for file_id in train.file_id:
    tffile = f"{GCS_PATH['asl-tfr']}/train/{file_id}.tfrecord"
    for batch in tf.data.TFRecordDataset([tffile], num_parallel_reads=tf.data.AUTOTUNE, compression_type='GZIP').map(decode_fn).take(2):
        print(list(batch)[1])
    break

## Get Dataset

In [ ]:
PAD = -100
def get_tfrec_dataset(tfrecords, batch_size=64, max_len=64, cache=True, drop_remainder=False, augment=False, shuffle=False, repeat=False):
    print("# Getting Dataset")
    ds = tf.data.TFRecordDataset(tfrecords, num_parallel_reads=tf.data.AUTOTUNE, compression_type='GZIP')
    ds = ds.map(decode_fn, tf.data.AUTOTUNE)
    ds = ds.map(convert_fn, tf.data.AUTOTUNE)
    ds = ds.map(lambda landmarks, phrase: preprocess(landmarks, phrase, augment=augment, max_len=max_len), tf.data.AUTOTUNE)

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(shuffle)
        options = tf.data.Options()
        options.experimental_deterministic = (False)
        ds = ds.with_options(options)

    if IS_TPU:
        ds = ds.padded_batch(batch_size,
                             padding_values=(tf.constant(PAD, dtype=tf.float32),
                                             tf.constant(PAD_TOKEN_IDX, dtype=tf.int32)),
                             padded_shapes=([FRAME_LEN, len(FEATURE_COL)], [64]),
                             drop_remainder=drop_remainder)
    else:
        ds = ds.batch(batch_size, drop_remainder=drop_remainder)

    df = ds.prefetch(tf.data.AUTOTUNE)

    if cache:
        ds = ds.cache()

    print("# Done")
    return ds

In [ ]:
print(f"Train: {len(TRAIN_FILENAMES)} TFRecord files.")
print(f"Train Supplemental: {len(TRAIN_FILENAMES_SUPPLEMENTAL)} TFRecord files.")

see the data

In [ ]:
dummy_d = get_tfrec_dataset(TRAIN_FILENAMES[-1:],
                            batch_size=CONFIG["batch_size"],
                            max_len=CONFIG["max_len"])

batch_test = next(iter(dummy_d))
print(list(batch_test)[0][0].shape, list(batch_test)[1][0])
INPUT_SHAPE = list(list(batch_test)[0][0].shape)
print(INPUT_SHAPE)

## Utils

In [ ]:
def CTCLoss(labels, logits):
    label_length = tf.reduce_sum(tf.cast(labels != PAD_TOKEN_IDX, tf.int32), axis=-1)
    if not IS_TPU:
        logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    else:
        logit_length = tf.ones(CONFIG['batch_size'], dtype=tf.int32) * tf.cast(64, tf.int32)
    loss = tf.nn.ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=PAD_TOKEN_IDX,
            logits_time_major=False
        )
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
def num_to_char_fn(y):
    return [ORD2CHAR.get(x, "") for x in y]

@tf.function()
def decode_phrase(pred):
    x = tf.argmax(pred, axis=1)
    diff = tf.not_equal(x[:-1], x[1:])
    adjacent_indices = tf.where(diff)[:, 0]
    x = tf.gather(x, adjacent_indices)
    mask = x != PAD_TOKEN_IDX
    x = tf.boolean_mask(x, mask, axis=0)
    return x

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    output_text = []
    for result in pred:
        result = "".join(num_to_char_fn(decode_phrase(result).numpy()))
        output_text.append(result)
    return output_text

In [ ]:
class CallbackEval(tf.keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset, freq=5):
        super().__init__()
        self.dataset = dataset
        self.freq = freq

    def on_epoch_end(self, epoch: int, logs=None):
        if (epoch + 1) % self.freq == 0:
            predictions = []
            targets = []
            for batch in self.dataset:
                X, y = batch
                batch_predictions = self.model(X)
                batch_predictions = decode_batch_predictions(batch_predictions)
                predictions.extend(batch_predictions)
                for label in y:
                    label = "".join(num_to_char_fn(label.numpy()))
                    targets.append(label)
            print("-" * 100)
            # for i in np.random.randint(0, len(predictions), 2):
            for i in range(32):
                print(f"Target    : {targets[i]}")
                print(f"Prediction: {predictions[i]}, len: {len(predictions[i])}")
                print("-" * 100)

In [ ]:
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print()
        gc.collect()
        k.clear_session()
        print("Cleared epoch session")

## Get Model

In [ ]:
#Copied from previous comp 1st place model: https://www.kaggle.com/code/hoyso48/1st-place-solution-training
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self,
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True

    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x


def TransformerBlock(dim=256, num_heads=4, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

#### Model

In [ ]:
def get_model(dim = 384, dropout_step=0):
    inp = tf.keras.Input(INPUT_SHAPE)
    x = inp
    ksize = 11
    x = tf.keras.layers.Masking(mask_value=PAD,input_shape=INPUT_SHAPE)(inp)
    x = tf.keras.layers.Dense(dim, use_bias=False,name='stem_conv')(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn')(x)

    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = TransformerBlock(dim,expand=2)(x)

    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = TransformerBlock(dim,expand=2)(x)
    x = TransformerBlock(dim,expand=2)(x)

#     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
#     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
#     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
#     x = TransformerBlock(dim,expand=2)(x)

    x = tf.keras.layers.Dense(dim*2,activation=None,name='top_conv')(x)
    x = tf.keras.layers.AveragePooling1D(2)(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(len(CHAR2ORD))(x)

    model = tf.keras.Model(inp, x)
    return model

In [ ]:
tf.keras.backend.clear_session()
model_test = get_model()
print("Data Shape (input, target)", batch_test[0].shape, batch_test[1].shape)
y_hat = model_test(batch_test[0])
print("Output Shape", y_hat.shape)
print("CTC Loss", CTCLoss(batch_test[1], y_hat))
model_test.summary()

Data Shape (input, target) (128, 128, 156) (128, 64)


Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.
Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


Output Shape (128, 64, 60)
CTC Loss tf.Tensor(195.8284, shape=(), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 156)]   0           []                               
                                                                                                  
 masking (Masking)              (None, 128, 156)     0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Dense)              (None, 128, 384)     59904       ['masking[0][0]']                
                                                                                                  
 stem_bn (BatchNormalization)   (None, 128, 384)     1536        ['stem_conv[0][0]']              
             

## Training

In [ ]:
TRAIN_CONFIG = {"print_transcription": True,
                "float16": False,
                "verbose": 1,
                "seeds": [42],
                "fold": 'part',
                "clear_memory": True}

In [ ]:
def train_fold(seed, train_files, valid_files, strategy=None):
    # set seed
    print("### Training with seed:", {seed}, " data: ", TRAIN_CONFIG['fold'])
    print("# Seeding")
    seed_everything(seed=seed)

    tf.keras.backend.clear_session()
    gc.collect()
    tf.config.optimizer.set_jit(True)

    if TRAIN_CONFIG['float16']:
        try:
            policy = tf.keras.mixed_precision.Policy('mixed_bfloat16')
            tf.keras.mixed_precision.set_global_policy(policy)
        except:
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
    else:
        policy = tf.keras.mixed_precision.Policy('float32')
        tf.keras.mixed_precision.set_global_policy(policy)

    if valid_files is not None:
        train_dataset = get_tfrec_dataset(train_files,
                                          batch_size=CONFIG["batch_size"],
                                          max_len=CONFIG["max_len"], cache=False,
                                          drop_remainder=True, augment=True,
                                          repeat=False, shuffle=False)
        valid_dataset = get_tfrec_dataset(valid_files,
                                          batch_size=CONFIG["batch_size"],
                                          max_len=CONFIG["max_len"], cache=False,
                                          drop_remainder=False, augment=False,
                                          repeat=False, shuffle=False)
    else:
        train_dataset = get_tfrec_dataset(train_files,
                                          batch_size=CONFIG["batch_size"],
                                          max_len=CONFIG["max_len"], cache=False,
                                          drop_remainder=False, augment=True,
                                          repeat=False, shuffle=False)
        valid_dataset = None

    with strategy.scope():
        print("# Getting model")
        model=get_model()

        schedule = OneCycleLR(CONFIG["lr"], CONFIG["epoch"],
                                  warmup_epochs=CONFIG["epoch"]*CONFIG["warmup"],
                                  steps_per_epoch=314, decay_epochs=CONFIG["epoch"],
                                  lr_min=CONFIG["lr_min"],
                                  decay_type='cosine',
                                  warmup_type='linear')
        decay_schedule = OneCycleLR((CONFIG["lr"])*CONFIG["weight_decay"], CONFIG["epoch"],
                                    warmup_epochs=CONFIG["epoch"]*CONFIG["warmup"],
                                    steps_per_epoch=314, decay_epochs=CONFIG["epoch"],
                                    lr_min=CONFIG["lr_min"]*CONFIG["weight_decay"],
                                    decay_type='cosine',
                                    warmup_type='linear')

        if CONFIG['optimizer'] == "ranger":
            optimizer = tfa.optimizers.RectifiedAdam(learning_rate=schedule, weight_decay=decay_schedule, sma_threshold=4)
            optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=5)
        elif CONFIG['optimizer'] == "adam":
            optimizer = keras.optimizers.Adam(learning_rate=schedule, weight_decay=CONFIG['weight_decay'])

        if CONFIG['loss'] == "cce":
            loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1)
        elif CONFIG['loss'] == "ctc":
            loss = CTCLoss

        print("# Compiling model")
        model.compile(optimizer=optimizer,
                      loss=loss,
                      run_eagerly=False)
        print("# Done")

    print("# Starting training")

    ## callbacks
    clear_memory = ClearMemory()
    logger = tf.keras.callbacks.CSVLogger(f'{root_save}/fold{seed}-logs.csv')
    sv_loss0 = tf.keras.callbacks.ModelCheckpoint(f'{root_save}/fold{seed}-best.h5', monitor='val_loss', verbose=0, save_best_only=True,
                                                  save_weights_only=True, mode='min', save_freq='epoch')
    sv_loss1 = tf.keras.callbacks.ModelCheckpoint(f'{root_save}/fold{seed}-best.h5', monitor='loss', verbose=0, save_best_only=True,
                                                  save_weights_only=True, mode='min', save_freq='epoch')
    validation_callback = CallbackEval(dummy_d.take(1), freq=3)

    callbacks=[logger]
    if valid_dataset is not None:
        callbacks.append(sv_loss0)
    else:
        callbacks.append(sv_loss1)

    if TRAIN_CONFIG["print_transcription"]:
        callbacks.append(validation_callback)
    if TRAIN_CONFIG["clear_memory"]:
        callbacks.append(clear_memory)

    history = model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=CONFIG["epoch"],
        # steps_per_epoch=steps_per_epoch,
        callbacks=callbacks,
        verbose=TRAIN_CONFIG['verbose'])

    try:
        model.load_weights(f'{root_save}/fold{seed}-best.h5')
    except:
        pass

    if valid_dataset is not None:
        val = model.evaluate(valid_dataset, verbose=TRAIN_CONFIG['verbose'])
    else:
        val = None

    del model, train_dataset, valid_dataset
    _ = gc.collect()
    return history, val

def run_training():
    history = {}
    val = {}
    for seed in TRAIN_CONFIG['seeds']:
        if TRAIN_CONFIG['fold'] == 'all':
            train_files = TRAIN_FILENAMES + TRAIN_FILENAMES_SUPPLEMENTAL[:int(len(TRAIN_FILENAMES_SUPPLEMENTAL)*CONFIG['supplemental_size'])]
            history_fold, val_fold = train_fold(seed=seed, train_files=train_files, valid_files=None, strategy=STRATEGY)
        elif TRAIN_CONFIG['fold'] == "part":
            len_train = int(len(TRAIN_FILENAMES) * (1 - CONFIG["valid_size"]))
            train_files = TRAIN_FILENAMES[:len_train] + TRAIN_FILENAMES_SUPPLEMENTAL[:int(len(TRAIN_FILENAMES_SUPPLEMENTAL)*CONFIG['supplemental_size'])]
            valid_files = TRAIN_FILENAMES[len_train:]
            history_fold, val_fold = train_fold(seed=seed, train_files=train_files, valid_files=valid_files, strategy=STRATEGY)

        history[seed] = history_fold
        val[seed] = val_fold

    return history, val

In [ ]:
history, val = run_training()

### Training with seed: {42}  data:  part
# Seeding
# Getting Dataset
# Done
# Getting Dataset
# Done
# Getting model
# Compiling model
# Done
# Starting training
Epoch 1/50
    286/Unknown - 1438s 5s/step - loss: 78.6927